# Работа с данными
Создан класс обработчик, который обрабатывает pcap - файлы , а именно протокол ARP и вытаскивает
timestamp – время, когда был захвачен пакет.
sender_mac – MAC-адрес устройства, отправляющего ARP-запрос или ответ.
sender_ip – IP-адрес отправителя.
target_mac – MAC-адрес устройства, которому предназначен ARP-запрос или ответ.
target_ip – IP-адрес получателя.
operation – тип ARP-операции (1 – запрос, 2 – ответ).
label – нормальный - 0 , аномальный - 1. Также можно вписать в label, как душе будет угодно для метки


In [2]:
from scapy.all import rdpcap, ARP
import pandas as pd

class ARPDataHandler:
    def __init__(self, file_paths, labels):
  
        self.file_paths = file_paths
        self.labels = labels
        self.data = []

    def process_files(self):
        # Считывает файлы pcapng и извлекает функции ARP
        for file_path, label in zip(self.file_paths, self.labels):
            packets = rdpcap(file_path)  
            self.extract_arp_features(packets, label)

    def extract_arp_features(self, packets, label):
        # Извлекает функции, связанные с ARP, из пакетов
        for packet in packets:
            if packet.haslayer(ARP):  
                arp_layer = packet[ARP]
                self.data.append({
                    "timestamp": packet.time, # Время захвата пакета
                    "sender_mac": arp_layer.hwsrc, # Mac-адресс отправляющего запрос
                    "sender_ip": arp_layer.psrc, # ip -адерсс отправляющего запрос
                    "target_mac": arp_layer.hwdst, # Mac-адресс которому предназначен запрос или ответ
                    "target_ip": arp_layer.pdst, # Ip получателя
                    "operation": arp_layer.op,  # 1 = запрос 2 = ответ
                    "label": label  #  где 0 нормальный трафик, 1 аномальный. Также по названи. в CSV файле можно понять.
                })

    def save_to_csv(self, output_file):
        # Преобразует данные в DataFrame и сохраняет их в виде CSV-файла
        df = pd.DataFrame(self.data)
        df.to_csv(output_file, index=False)
        print(f"Processed {len(df)} ARP packets. Data saved to {output_file}")

# Путь к файлам
file_paths = [
    r'D:\Проекты\Дипломаня работа\DoFitN\Data\PCAP\SANYA\Normal.pcapng',  
    r'D:\Проекты\Дипломаня работа\DoFitN\Data\PCAP\SANYA\AnomalS.pcapng'  

]

labels = 0,1


arp_handler = ARPDataHandler(file_paths, labels)
arp_handler.process_files()
arp_handler.save_to_csv("New_data_sanya.csv")


Processed 3667 ARP packets. Data saved to New_data_sanya.csv


# Аугментация данных ,для увеелчения датасета


In [3]:
from scapy.all import rdpcap, wrpcap, ARP
import random
import pandas as pd

# Функция для генерации случайного MAC-адреса
def random_mac():
    return ":".join(f"{random.randint(0x00, 0xFF):02x}" for _ in range(6))

# Функция для генерации случайного IP-адреса (в пределах локальной сети)
def random_ip():
    return f"192.168.1.{random.randint(1, 254)}"

# Загрузка аномального трафика
anomalous_packets = rdpcap(r'D:\Проекты\Дипломаня работа\DoFitN\Data\arp_A2.pcapng')

# Создаем список для новых аномальных пакетов
augmented_packets = []

for pkt in anomalous_packets:
    if pkt.haslayer(ARP):
        new_pkt = pkt.copy()
        
        # Генерируем небольшие изменения в пакетах
        if random.random() > 0.5:  # 50% пакетов меняем MAC-адрес
            new_pkt.hwsrc = random_mac()
            new_pkt.hwdst = random_mac()
        
        if random.random() > 0.5:  # 50% пакетов меняем IP-адрес
            new_pkt.psrc = random_ip()
            new_pkt.pdst = random_ip()
        
        augmented_packets.append(new_pkt)

# Сохраняем новый датасет с аугментированными данными
wrpcap("arp_A2_augmented.pcapng", augmented_packets)

print(f"Создан новый файл с аугментированными данными: arp_A_augmented.pcapng")


Создан новый файл с аугментированными данными: arp_A_augmented.pcapng
